# 01 Preparación de datos (QC + splits + tasks.json)

Este notebook prepara los splits `train/val/test` a partir de los logs de Udacity y genera `tasks.json` que usa el pipeline continual.

__Objetivos__:

- Cargar y normalizar rutas del `driving_log.csv`.
- Verificar existencia de imágenes.
- Hacer split estratificado por bins de `steering` (reduce sesgo a recta).
- Escribir los CSV resultantes y `tasks.json`.

__Cómo encaja en el TFM__: deja los datos listos y reproducibles para los experimentos de aprendizaje continual.

__Imports y paths__

Importamos el módulo de preparación (`src/prep/data_prep.py`) y fijamos rutas base. Esta celda no ejecuta lógica de negocio, solo prepara utilidades y paths.

In [13]:
# -*- coding: utf-8 -*-
"""Prepara splits (train/val/test) y genera tasks.json para el pipeline continual.

Qué hace:
1) Carga y normaliza el driving_log.csv de cada recorrido (RUNS).
2) Normaliza rutas (soporta barras invertidas y recorte a 'IMG/...').
3) Filtra filas sin imágenes (center/left/right).
4) Split estratificado por bins de 'steering'.
5) Guarda:
   - data/processed/<run>/canonical.csv
   - data/processed/<run>/{train,val,test}.csv
   - data/processed/tasks.json
"""

from pathlib import Path
import sys, json
import pandas as pd

# Raíz del repo (permite ejecutar notebooks desde notebooks/ o raíz)
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# API de preparación offline
from src.prep.data_prep import PrepConfig, run_prep, verify_processed_splits  # <- usa tu módulo
RAW  = ROOT / "data" / "raw" / "udacity"
PROC = ROOT / "data" / "processed"


__Parámetros de la preparación__

Definimos __qué recorridos__ preparar y los __híperparámetros__ del split. Aquí __NO__ balanceamos ni expandimos cámaras: este notebook es QC + splits básicos.

In [14]:
# Parámetros de la preparación
RUNS = ["circuito1", "circuito2"]

CFG = PrepConfig(
    root=ROOT,
    runs=RUNS,
    use_left_right=False,  # QC + split básico (sin expansión)
    steer_shift=0.2,       # no aplica si use_left_right=False, lo mantenemos por consistencia
    bins=21,               # nº de bins para estratificar por 'steering'
    train=0.70,            # proporción train
    val=0.15,              # proporción val (test se infiere)
    seed=42,               # reproducibilidad
    target_per_bin="auto", # (no se usa aquí; el balanceo es en 01A_PREP_BALANCED)
    cap_per_bin=12000,     # (no se usa aquí)
)
CFG


PrepConfig(root=PosixPath('/home/cesar/proyectos/TFM_SNN'), runs=['circuito1', 'circuito2'], use_left_right=False, steer_shift=0.2, bins=21, train=0.7, val=0.15, seed=42, target_per_bin='auto', cap_per_bin=12000)

__Ejecutar preparación y verificar__

Ejecutamos `run_prep(CFG)`, que escribe `canonical.csv`, `train/val/test.csv` y `tasks.json`. Luego validamos que existan los CSV requeridos.

In [15]:
# Ejecutar prep + verificar
manifest = run_prep(CFG)
print("OK:", PROC/"prep_manifest.json")

# Verificación: existen train/val/test por run
verify_processed_splits(PROC, RUNS)
print("OK: splits 'train/val/test' encontrados.")


OK: /home/cesar/proyectos/TFM_SNN/data/processed/prep_manifest.json
OK: splits 'train/val/test' encontrados.


__Resumen rápido__

Presentamos una tabla con las rutas de los splits por cada run (informativo/diagnóstico).

In [16]:
# Resumen de rutas de splits (visualización)
tasks_json = json.loads((PROC/"tasks.json").read_text(encoding="utf-8"))
pd.DataFrame({
    "run": tasks_json["tasks_order"],
    "train_csv": [tasks_json["splits"][r]["train"] for r in tasks_json["tasks_order"]],
    "val_csv":   [tasks_json["splits"][r]["val"]   for r in tasks_json["tasks_order"]],
    "test_csv":  [tasks_json["splits"][r]["test"]  for r in tasks_json["tasks_order"]],
})


,run,train_csv,val_csv,test_csv
0,circuito1,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
1,circuito2,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...,/home/cesar/proyectos/TFM_SNN/data/processed/c...
